# This notebook contains the analysis of the MC information truth of Tl source 

In [ ]:
import os
import sys
import random
import tables as tb
import numpy  as np

import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from   invisible_cities.database               import load_db

import invisible_cities.reco.paolina_functions as plf
from   invisible_cities.reco.dst_functions     import load_xy_corrections

from   invisible_cities.io.dst_io              import load_dst
from   invisible_cities.io.hits_io             import load_hits
from   invisible_cities.io.hits_io             import load_hits_skipping_NN
from invisible_cities.io.mchits_io             import load_mchits

from   invisible_cities.types.ic_types         import xy
from   invisible_cities.types.ic_types         import NN

from   invisible_cities.evm.event_model        import Cluster, Hit

import invisible_cities.core.fit_functions     as fitf

from   invisible_cities.icaro.hst_functions    import hist
from   invisible_cities.icaro.hst_functions    import gausstext

from   invisible_cities.database               import load_db

In [ ]:
def merge_NN_hits(hits_all,hits_nonNN):

    # Iterate through the nonNN dictionary and update the energies including the NN hits from the "all" dictionary.
    for (evt,hc) in hits_nonNN.items():
        # Get the corresponding collection of all hits.
        hc_all = hits_all[evt]            
        # Add energy from all NN hits to hits in closest slice.
        for h1 in hc_all.hits:
            if(h1.Q == NN):
                # Find the hits to which the energy will be added.
                zdist_min = -1
                h_add = []
                for h2 in hc.hits:
                    zdist = np.abs(h1.Z - h2.Z)
                    if(zdist_min < 0 or zdist < zdist_min):
                        zdist_min = zdist
                        h_add = []
                        h_add.append(h2)
                    elif(zdist == zdist_min):
                        h_add.append(h2)

                # Add the energy.
                hadd_etot = sum([ha.E for ha in h_add])
                for ha in h_add:
                    ha.energy += h1.E*(ha.E/hadd_etot)

### Parameters

In [ ]:
DetGeo = load_db.DetectorGeo()
fwhmQbb = 0.006
vox_size = np.array([15,15,15],dtype=np.int16)    # voxel size
blob_radius = 21.   # blob radius in mm
file_base_name = 'tl_pmaps_7bar_NEXT_v1_00_05_v0.9.2_20171011_krmc_tl'
directory_name = '/home/Tl_v0.9.2'
start = 0 ## analyzes first file 0
numb = 5 ## number of files to be analyzed

### Containers for variables to be saved to file

In [ ]:
evt_energies = []
trk_energies = []
number_of_tracks = []
number_of_voxels = []
number_of_voxels_main = []
length_main = []
energy_main = []
l_eblob1 = []; l_eblob2 = []

### Loop on the files

In [ ]:
for n in range(start,start+numb):
    lhits = []
    hits_file = '{0}/{1}_hits.{2}.h5'.format(directory_name, file_base_name, n)
    print('Analyzing {0}'.format(hits_file))

    hit_dict = load_mchits(hits_file)

    # Correct the energy of the hits, one by one
    hitc_cevt = {}
    for evt, hits in hit_dict.items():
        
        mc_evt_energy = sum([h.E for h in hits])
        sigmaE = fwhmQbb * np.sqrt(2.458)*np.sqrt(mc_evt_energy)/2.35
        smeared_evt_energy = mc_evt_energy + random.gauss(0, 1) * sigmaE
        conv_factor = smeared_evt_energy/mc_evt_energy

        hc_smeared = []
        for hit in hits:
        # Check that only the hits of the ACTIVE volume are considered
            if hit.Z < DetGeo.ZMAX[0] and hit.X < DetGeo.XMAX[0] and hit.X > DetGeo.XMIN[0]:
                hsmeared = Hit(0,Cluster(0, xy(hit.X,hit.Y), xy(0,0), 0), hit.Z, hit.E*1000.*conv_factor)
                hc_smeared.append(hsmeared)
        hitc_cevt[evt] = hc_smeared

    # Build tracks and find blobs
    for nevt, hitc in hitc_cevt.items():

        tot_e = sum([hh.E for hh in hitc])
        evt_energies.append(tot_e)
                       
        voxels = plf.voxelize_hits(hitc,vox_size)
        trks = plf.make_track_graphs(voxels,vox_size)
        number_of_tracks.append(len(trks))
    
        energy_tr = []
        for t in trks:
            if(len(t.nodes()) < 1):
                etrk = 0
            else:
                etrk = sum([vox.E for vox in t.nodes()])
                energy_tr.append(etrk)
                trk_energies.append(etrk)
                number_of_voxels.append(len(t.nodes()))
   
                if len(trks) == 0:
                    continue
        
                itmax = np.argmax(energy_tr)
    
                number_of_voxels_main.append(len(trks[itmax].nodes()))
                e_main = sum([vox.E for vox in trks[itmax].nodes()])
                energy_main.append(e_main)

                if e_main < 1576. or e_main > 1607.:
                    continue

                Eblob1, Eblob2 = plf.blob_energies(trks[itmax],blob_radius)
                               
                # Ensure blob1 always has higher energy
                if(Eblob2 > Eblob1):
                    eswap = Eblob1
                    Eblob1 = Eblob2
                    Eblob2 = eswap
              
                l_eblob1.append(Eblob1)
                l_eblob2.append(Eblob2)
                length_main.append(plf.length(trks[itmax]))

In [ ]:
a_eblob1 = np.array(l_eblob1)
a_eblob2 = np.array(l_eblob2)

a_evt_energies = np.array(evt_energies)
a_trk_energies = np.array(trk_energies)
a_number_of_tracks = np.array(number_of_tracks)
a_number_of_voxels = np.array(number_of_voxels)
a_number_of_voxels_main = np.array(number_of_voxels_main)
a_length_main = np.array(length_main)
a_energy_main = np.array(energy_main)

evt_file = '{0}/analysis_{1}_truth'.format(directory_name, file_name)
np.savez(evt_file, a_eblob1=a_eblob1, a_eblob2=a_eblob2, a_evt_energies=a_evt_energies, a_trk_energies=a_trk_energies, a_number_of_tracks=a_number_of_tracks, a_number_of_voxels=a_number_of_voxels, a_number_of_voxels_main=a_number_of_voxels_main, a_length_main=a_length_main, a_energy_main=a_energy_main) 